In [25]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle
from os.path import join as oj
from tqdm import tqdm
import math
import sys
import pandas as pd
sys.path.append('../../')
from copy import deepcopy
from functions import load_usafacts_data
import helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
data_dir = '../..'
df_covid = load_usafacts_data.load_daily_data(dir_mod='../..')
with open(oj(data_dir, 'data/df_county_level_cached.pkl'), 'rb') as f:
    county_df = pickle.load(f)
# county_df = load_data.load_county_level(dir_mod='..')
with open("countyFIPS_to_city.pkl", 'rb') as f:
    countyFIPS_to_city = pickle.load(f)
contact_info = pd.read_csv("all_state_numbers.csv", index_col = 0)

In [19]:
idxs_used = [] # keeps track of which row indexes in the contact_info were matched
def find_contact(entry, contact_info, idx):
    '''Searches the entry in county_df to see if it is contained in the contact info
    '''
    same_state = contact_info[contact_info["state"] == entry["StateNameAbbreviation"]]
    same_county = same_state[same_state["public_health"].apply(lambda x: entry["CountyName"] in x)]
    if same_county.shape[0] >= 1:
        idxs_used.append(idx)
        return ", ".join(same_county["phone_number"]), ", ".join(same_county["public_health"])
    elif entry["countyFIPS"] in countyFIPS_to_city:
        city = countyFIPS_to_city[entry["countyFIPS"]]
        try:
            same_city = same_state[same_state["public_health"].apply(lambda x: city in x)]
            if same_city.shape[0] >= 1:
                idxs_used.append(idx)
                return ", ".join(same_city["phone_number"]), ", ".join(same_city["public_health"])
            else:
                return None, None
        except:
            return None, None
    else:
        return None, None

In [ ]:
health_dept_names = []
contact_numbers = []
for idx, county in tqdm(county_df.iterrows()):
    result = find_contact(county, contact_info, idx)
    health_dept_names.append(result[1])
    contact_numbers.append(result[0])
df = deepcopy(county_df)
df["HealthDeptName"] = health_dept_names
df["HealthDeptContact"] = contact_numbers

In [21]:
!wget https://github.com/Yu-Group/covid-19-ventilator-demand-prediction/blob/4bd43c43d1068f0ccb9eb35725b4897ea408f147/scraping/countyFIPS_to_city.pkl

--2020-03-28 09:43:22--  https://github.com/Yu-Group/covid-19-ventilator-demand-prediction/blob/4bd43c43d1068f0ccb9eb35725b4897ea408f147/scraping/countyFIPS_to_city.pkl
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘countyFIPS_to_city.pkl’

countyFIPS_to_city.     [ <=>                ]  65.20K   425KB/s    in 0.2s    

2020-03-28 09:43:22 (425 KB/s) - ‘countyFIPS_to_city.pkl’ saved [66764]



In [29]:
df = pd.merge(df, df_covid, on='countyFIPS', how='outer')

In [30]:
matched_rows = ~df["HealthDeptContact"].isna()
nofips_rows = [k for k in np.arange(contact_info.shape[0]) if not k in idxs_used]

key_sort = 'tot_deaths' # tot_deaths, StateName
df_matched = df[matched_rows].sort_values(by=key_sort, ascending=False)
df_nocontact = df[~matched_rows].sort_values(by=key_sort, ascending=False)
df_nocontact["HealthDeptName"] = ""
df_nocontact["HealthDeptContact"] = ""
df_nofips = contact_info.iloc[nofips_rows].rename(columns={'public_health': 'HealthDeptName', 'phone_number': 'HealthDeptContact'})
for k in ['CountyName', 'countyFIPS']:
    df_nofips[k] = ''
df_nofips['StateName'] = df_nofips['state']

In [31]:
print('matched', df_matched.shape[0], 'nocontact', df_nocontact.shape[0],
      'nofips', df_nofips.shape[0])

matched 2334 nocontact 786 nofips 1081


# match the nofips data

In [49]:
oracleFIPS = helper.get_fips_df()
oracleFIPS.head(1)

,countyFIPS,state,CountyName
0,01001,AL,Autauga


In [35]:
# Let's Extract the county name
df_nofips_tmp = df_nofips
health_names = df_nofips_tmp["HealthDeptName"]
health_names = [x.split("County")[0] for x in health_names]
health_names = [str.rstrip(x) if "Health" not in x else np.nan for x in health_names]
df_nofips_tmp["CountyName"] = health_names
df_nofips_tmp = df_nofips_tmp.drop("countyFIPS", axis=1)
df_nofips_tmp = df_nofips_tmp.merge(oracleFIPS, on=["CountyName", "state"], how="left")

# Finalize 
df_nofips = df_nofips_tmp
df_nofips_matched = df_nofips[df_nofips["countyFIPS"].notnull()]
df_nofips_unmatched = df_nofips[df_nofips["countyFIPS"].isnull()]

print("Matched %s previously nofips; %s still unmatched" % (len(df_nofips_matched), len(df_nofips_unmatched)))

Matched 733 previously nofips; 348 still unmatched


In [40]:
df_matched.head()

,CountyName,StateName,countyFIPS,HealthDeptName,HealthDeptContact,tot_deaths
1831,New York,New York,36061,New York City Department of Health and Mental ...,13473964131,365.0
2941,King,Washington,53033,Public Health - Seattle and King County,12062964600,109.0
1121,Orleans,Louisiana,22071,City of New Orleans Department of Health,15046582500,46.0
1285,Wayne,Michigan,26163,Wayne County Health Department,13132240810,26.0
1852,Suffolk,New York,36103,Suffolk County Department of Health Services,16318540100,22.0


In [ ]:
df_nofips_matched = pd.merge(df_nofips_matched, df_covid, on='countyFIPS', how='left')

In [ ]:
df_tot = pd.merge(df_matched, df_nofips_matched, on='countyFIPS', how='outer')

In [ ]:
# We only need these columns, let's append them
df_matched = df_matched[['CountyName', 'StateName', 'countyFIPS', 'HealthDeptName', 'HealthDeptContact', 'tot_deaths']]
df_nofips_matched = df_nofips_matched[['CountyName', 'StateName', 'countyFIPS', 'HealthDeptName', 'HealthDeptContact', 'tot_deaths']]
df_matched = df_matched.append(df_nofips_matched)
# print('matched', df_matched.shape[0], 'nocontact', df_nocontact.shape[0],
#       'nofips', df_nofips.shape[0])
# Update df_nofips to be currently unmatched
df_nofips = df_nofips_unmatched

# upload to gsheets

In [17]:
import pygsheets
gc = pygsheets.authorize(service_file='../creds.json')

#open the google spreadsheet (where 'PY to Gsheet Test' is the name of my sheet)
sheet_name = 'Contact info'
sh = gc.open(sheet_name) # name of the hospital

In [18]:
wks = sh[1]
wks.update_value('A1', "County-level contact information, scraped from here: https://www.naccho.org/membership/lhd-directory")
# wks.update_value('A2', "Columns A-E are read-only")
wks.set_dataframe(df_matched[['CountyName', 'StateName', 'countyFIPS', 'HealthDeptName', 'HealthDeptContact']], (5, 1))

In [ ]:
# wks = sh[2]
# wks.update_value('A1', "County-level contact information for counties missing a contact #")
# wks.update_value('A2', "Columns A-C are read-only")
# wks.set_dataframe(df_nocontact[['CountyName', 'StateName', 'countyFIPS', 'HealthDeptName', 'HealthDeptContact']], (5, 1))

In [93]:
# wks = sh[6]
# wks.update_value('A1', "County-level contact information (not linked to a county)")
# wks.update_value('A2', "Columns A-B are read-only")
# wks.set_dataframe(df_nofips[['HealthDeptName', 'HealthDeptContact', 'StateName', 'CountyName', 'countyFIPS']], (5, 1))